In [199]:
from tensorflow.python.keras.models import load_model
model = load_model('dnn_regression.h5')

In [200]:
import pandas as pd
import folium
import numpy as np
import math
# 시각화 라이브러리
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import seaborn as sns
%matplotlib inline

In [203]:
df = pd.read_csv('C:/Users/doong/Desktop/PythonWorkspace/창종설/df_new.csv')

In [204]:
df.keys()

Index(['시설명', '시간대구분', '주소', '시설종류', '구역지정수', 'CCTV 수', '도로표지판 수', '과속방지턱 수',
       '위도', '경도', 'CCTV설치여부', '보호구역도로폭', 'tmp', '속도', '도로명', '행정동', '시군구명',
       '총생활인구수', '사망자수', '중상자수', '경상자수', '부상신고자수', '가해운전자 차종', '피해운전자 차종',
       '피해운전자 연령', '사고 수', '통행량 도로명', '통행량', '면적', '인구밀도', '위험도',
       'cluster_label', '위험도군집', '일출/일몰', '등하교시간', '점심시간', '가해차종엔진125cc이상',
       '피해자어린이유무', '피해자교통수단탑승유무', '보호구역초등학교여부', '활동시간', '1차선이하', '표지판8개이하',
       '방지턱43개이하', '구역지정2개이하', 'CCTV3개이하', '피해자초고령여부', '가해차종4륜유무',
       '피해자연령38세이하', '피해차량승합차량여부', '일방통행여부', '방지턱16개이하', 'CCTV10개이하',
       '초등학교셔틀버스운영여부'],
      dtype='object')

In [205]:
df_predict = df.drop(['시설명','주소','위도', '경도','보호구역도로폭','tmp',
          '도로명', '행정동', '시군구명', '총생활인구수', '사망자수', '중상자수', '경상자수', '부상신고자수',
          '가해운전자 차종', '피해운전자 차종', '피해운전자 연령', '사고 수', '통행량 도로명', '통행량', '면적',
          '인구밀도','cluster_label', '위험도군집','초등학교셔틀버스운영여부',
          '표지판8개이하', '방지턱43개이하', '구역지정2개이하', 'CCTV3개이하', '방지턱16개이하',
          'CCTV10개이하'],axis=1)

In [206]:
df_predict.keys()

Index(['시간대구분', '시설종류', '구역지정수', 'CCTV 수', '도로표지판 수', '과속방지턱 수', 'CCTV설치여부',
       '속도', '위험도', '일출/일몰', '등하교시간', '점심시간', '가해차종엔진125cc이상', '피해자어린이유무',
       '피해자교통수단탑승유무', '보호구역초등학교여부', '활동시간', '1차선이하', '피해자초고령여부', '가해차종4륜유무',
       '피해자연령38세이하', '피해차량승합차량여부', '일방통행여부'],
      dtype='object')

In [207]:
X=pd.get_dummies(df_predict.iloc[:,[i if i<8 else i+1 for i in range(len(df_predict.keys())-1)]])
Y=df_predict['위험도']
X.describe()

X["시설종류_유치원"]=X["시설종류_유치원(병설)"]+X["시설종류_유치원(일반)"]
X["시설종류_특수학교"] = 0
X["시설종류_학원"] = 0
X["시설종류_기타"] = 0
X.describe()

X.drop(["시설종류_유치원(병설)","시설종류_유치원(일반)"],axis=1,inplace=True)

In [208]:
X = X.reindex(columns=['시간대구분', '구역지정수', 'CCTV 수', '도로표지판 수', '과속방지턱 수', 'CCTV설치여부', '속도',
       '일출/일몰', '등하교시간', '점심시간', '가해차종엔진125cc이상', '피해자어린이유무', '피해자교통수단탑승유무',
       '보호구역초등학교여부', '활동시간', '1차선이하', '피해자초고령여부', '가해차종4륜유무', '피해자연령38세이하',
       '피해차량승합차량여부', '일방통행여부', '시설종류_기타', '시설종류_어린이집', '시설종류_유치원', '시설종류_초등학교',
       '시설종류_특수학교', '시설종류_학원'])
X.keys()

Index(['시간대구분', '구역지정수', 'CCTV 수', '도로표지판 수', '과속방지턱 수', 'CCTV설치여부', '속도',
       '일출/일몰', '등하교시간', '점심시간', '가해차종엔진125cc이상', '피해자어린이유무', '피해자교통수단탑승유무',
       '보호구역초등학교여부', '활동시간', '1차선이하', '피해자초고령여부', '가해차종4륜유무', '피해자연령38세이하',
       '피해차량승합차량여부', '일방통행여부', '시설종류_기타', '시설종류_어린이집', '시설종류_유치원', '시설종류_초등학교',
       '시설종류_특수학교', '시설종류_학원'],
      dtype='object')

In [209]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시간대구분          12 non-null     int64  
 1   구역지정수          12 non-null     int64  
 2   CCTV 수         12 non-null     int64  
 3   도로표지판 수        12 non-null     float64
 4   과속방지턱 수        12 non-null     int64  
 5   CCTV설치여부       12 non-null     int64  
 6   속도             12 non-null     float64
 7   일출/일몰          12 non-null     int64  
 8   등하교시간          12 non-null     int64  
 9   점심시간           12 non-null     int64  
 10  가해차종엔진125cc이상  12 non-null     int64  
 11  피해자어린이유무       12 non-null     int64  
 12  피해자교통수단탑승유무    12 non-null     int64  
 13  보호구역초등학교여부     12 non-null     int64  
 14  활동시간           12 non-null     int64  
 15  1차선이하          12 non-null     int64  
 16  피해자초고령여부       12 non-null     int64  
 17  가해차종4륜유무       12 non-null     int64  
 18  피해자연령38세이하  

In [230]:
def change(x_v2):
    x_imp=x_v2.copy()
    idx_imp = [15,20]
    for i in range(len(x_imp)):
        x_imp.iloc[i,idx_imp[0]] = 0
        x_imp.iloc[i,idx_imp[1]] = 0
        # x_imp.iloc[i,idx_imp[1]] = x_imp.iloc[i,idx_imp[1]] + change * (99/x_imp.iloc[i,-7])
        # x_imp.iloc[i,idx_imp] = x_imp.iloc[i,idx_imp[1]] if x_imp.iloc[i,idx_imp[1]] < 1 else 1
    
    return x_imp

In [231]:
print(model.predict(change(X)))

[[ 1.2853081]
 [ 1.6477863]
 [ 3.203183 ]
 [ 1.4042306]
 [ 1.2812741]
 [ 1.4022413]
 [11.787932 ]
 [ 1.4343243]
 [ 2.1424499]
 [ 1.2475199]
 [ 9.4245825]
 [ 4.9978247]]


In [217]:
for i in range(17):
    print(i)
    print(model.predict(change(X,i)))

0
[[1.4042306]
 [1.7265719]
 [3.203183 ]
 [1.2242253]
 [1.391368 ]
 [1.6142707]
 [9.320072 ]
 [1.5131099]
 [2.1424499]
 [1.3263056]
 [9.4245825]
 [4.9978247]]
1
[[1.4042306]
 [1.6775296]
 [2.8351097]
 [1.4042306]
 [1.3423256]
 [1.6146063]
 [9.151474 ]
 [1.4640675]
 [2.1521966]
 [1.277263 ]
 [9.269932 ]
 [4.629547 ]]
2
[[1.4042306]
 [1.6284871]
 [2.4789736]
 [1.4042306]
 [1.2932831]
 [1.5655638]
 [8.982877 ]
 [1.415025 ]
 [2.1619432]
 [1.2282207]
 [9.11528  ]
 [4.2612705]]
3
[[1.4042306]
 [1.5794446]
 [2.3093507]
 [1.4042306]
 [1.2442406]
 [1.5165213]
 [8.81428  ]
 [1.3659825]
 [2.17169  ]
 [1.4042306]
 [8.9606285]
 [3.8929923]]
4
[[1.4042306]
 [1.5304022]
 [2.1877754]
 [1.4042306]
 [1.3257525]
 [1.4674789]
 [8.645682 ]
 [1.3169401]
 [2.1814365]
 [1.4042306]
 [8.805979 ]
 [3.5806987]]
5
[[1.4042306]
 [1.4813597]
 [2.066102 ]
 [1.4042306]
 [1.4042306]
 [1.4184364]
 [8.477084 ]
 [1.2678976]
 [2.191183 ]
 [1.4042306]
 [8.651143 ]
 [3.276153 ]]
6
[[1.4042306]
 [1.4323173]
 [1.9661219]
 [1.4